In [ ]:
!curl https://raw.githubusercontent.com/APSV-UPM/BusinessIntelligence/main/data/data_clean.csv > data.csv

# Machine Learning in Python
Machine Learning is the study and application of algorithms that generate statistic models that are able to perform a specific task without being specifically coded. Models are created by observing a set of samples and extracting and learning the patterns in the samples data.

The most used library in Python to work with ML is SciKit-Learn https://scikit-learn.org/stable/. This module includes the implementations of dozens of different ML algorithms and the functions needed to handle and transform the data that will be used by the ML models.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pandas as pd

Let's load our well known dataset of train events:

In [ ]:
df = pd.read_csv("data_clean.csv").dropna()
df.date = pd.to_datetime(df.date)

We are going to transform this train event data set into a table in which each row contains the information for a single train trip. These rows will indicate the number of packages that were transported on the train, the train used and the trip duration.

In [ ]:
t = df.groupby("trip_id").agg({"train_id":"first", "date":{'min', 'max'}, "cargo":'max'})
t.columns.map("_".join)

In [ ]:
trip_summary = df.groupby("trip_id").agg({"train_id":"first", "date":{'min', 'max'}, "cargo":'max'})
trip_summary.columns = trip_summary.columns.map("_".join)
trip_summary.columns = trip_summary.rename(columns={"date_min":"start_date", "date_max":"end_date", "train_id_first":"train_id", "cargo_max":"cargo"}).columns
trip_summary.reset_index(inplace=True, drop=True)
trip_summary["trip_duration"] = (trip_summary.end_date - trip_summary.start_date).dt.total_seconds()
trip_summary

Most ML algorithms can only work with numerical values, therefore, we have to convert the columns that contain text (categorical variables) into numbers. The simplest way to do this is by using a LabelEncoder, which just assigns each category to which it finds an integer.

In [ ]:
from sklearn import preprocessing
train_id_encoder = preprocessing.LabelEncoder().fit(trip_summary.train_id.unique())

for i in range(len(train_id_encoder.classes_)):
    print(f"""{train_id_encoder.classes_[i]} encoded as {i}""")
trip_summary["train_id_encoded"] = pd.Series(train_id_encoder.transform(trip_summary.train_id.values), index = trip_summary.index)

### Train and test
The main idea behind machine learning is that the algorithm will observe a training set of samples, which we should be sure to be representative of the data with which we are working with which. From this training set or training data, the algorithm will learn the patterns that describe the data and generates a model which can replicate the behavior shown on them. Then, using a *different* set of samples, which we call test set or test data, we will evaluate how accurate is the generated model, basically, by measuring the difference between the model output and the correct answer. It is very important to do not use the same data to train and test the model.

Let’s see it with an example.
We will use a popular dataset in ML called iris dataset. This dataset contains a series of measures of flowers and the type of the flower (for more info, check https://archive.ics.uci.edu/ml/datasets/iris). As we will see later, the task of the model we are going to create is a classification task. 
The dataset is made up of observations that contain 4 numeric variables and 1 categorical target. The task of the model is to predict the target of a sample by observing its numeric variables.


In [ ]:
# This example is based on one example from the scikit learn documentation 
# https://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html
from sklearn import neighbors, datasets
from sklearn import model_selection

x_feature = 0
y_feature = 1

n_neighbors = 5

iris = datasets.load_iris()

# In this notebook, we will always use x as the feature vector and y as the target
X = iris.data
y = iris.target

x_train, x_test, y_train, y_test = model_selection.train_test_split(X, y, test_size= 0.3)

clf = neighbors.KNeighborsClassifier(n_neighbors)
clf.fit(x_train, y_train)
y_out = clf.predict(x_test)

fig,ax = plt.subplots(figsize=(9, 8))

ax.scatter(x_train[:, x_feature], x_train[:, y_feature], c=y_train, cmap="plasma", edgecolors='k')
ax.scatter(x_test[:, x_feature], x_test[:, y_feature], c=y_out, cmap="plasma", marker = "x")
ax.scatter(x_test[:, x_feature], x_test[:, y_feature], c=y_test, cmap="plasma", alpha=0.3, marker = "o", s=200, edgecolors='k')
ax.scatter(x_test[y_out != y_test, x_feature], x_test[y_out != y_test, y_feature], facecolors='none', edgecolors='r', s=200)


ax.set_title("3-Class classification (k = %i)" % (n_neighbors))
plt.show()

print("Model accuracy: {:.2f}%".format(np.average(y_out==y_test)*100))

Although the dataset has 4 dimensions, the plot only shows 2 dimensions, which is easier for humans. The colors in the plot are the different types of flowers. Dots are the training data and cross the test data, the background of each test sample is the correct type, and the sample is surrounded with a red circle if it was misclasified. The accuracy is just the average of the correct classifications made by the model.

### K-Fold Cross-Validation
Ensure that the partition of the data used to train the model is representative may not be an easy task. For example, in the previous dataset it could be that the data we select to train don't contain samples of one of the class, or may be that the test data were particularly easy to classify.

Cross Validation is a group of techniques that solve this problem, all of them are based on the idea of performing several trainings and testings using different partitions each time. The most basic, and most popular, is the K-Fold Cross Validation.
In K-Fold, we choose a number k (usually from 5 to 10), then we divide our dataset into k folds. We will train the model k times, in each iteration we will use k-1 folds as the train data and the remaining fold as test data. The evaluation of the final model is calculated as the average of the evaluations for each iteration.

In [ ]:
from sklearn.model_selection import KFold
k = 3
kf = KFold(n_splits=k,shuffle=True)
accuracies = []
i=0
for train, test in kf.split(X): # train and test are the indices of the samples that will be used in each set
    i+=1
    x_train, x_test, y_train, y_test = X[train], X[test], y[train], y[test]
    clf = neighbors.KNeighborsClassifier(n_neighbors)
    clf.fit(x_train, y_train)
    y_out = clf.predict(x_test)


    fig,ax = plt.subplots(figsize=(9, 8))
    ax.scatter(x_train[:, x_feature], x_train[:, y_feature], c=y_train, cmap="plasma", edgecolors='k')
    ax.scatter(x_test[:, x_feature], x_test[:, y_feature], c=y_out, cmap="plasma", marker = "x")
    ax.scatter(x_test[:, x_feature], x_test[:, y_feature], c=y_test, cmap="plasma", alpha=0.3, marker = "o", s=200, edgecolors='k')
    ax.scatter(x_test[y_out != y_test, x_feature], x_test[y_out != y_test, y_feature], facecolors='none', edgecolors='r', s=200)
    ax.set_title("3-Class classification (k = %i) iteration %i" % (n_neighbors,i))
    plt.show()
    accuracy = np.average(y_out==y_test)*100
    accuracies.append(accuracy)
    print("Iteration {}: Model accuracy: {:.2f}%".format(i,accuracy))
    print(y_test)
    print(y_out)
print("Global accuracy: {:.2f}%".format(np.average(accuracies)))

## Regression
Regression is one of the possible tasks for an ML model. The objective is to predict a numeric output based on the input. One example of this could be to predict the number of products that will be sold based on the previous sales or calculate the probability that the stock market will grow tomorrow.


The most basic example of this task is to obtain the relation between two variables.

In [ ]:
from sklearn import datasets
import math

x, y = datasets.make_regression(n_samples=30, n_features=1, noise=0)
# try to modify the relation between x and y or play with the noise value
y = y**2
#y = np.sin(y)
fig,ax = plt.subplots(figsize=(9, 8))
ax.scatter(x, y)
plt.show()

Problems in the real world are more complicated, the target (variable we want to predict) usually depends on more variables (features).

### Some popular algorithms
There are many different algorithms or models that can be used for regression tasks. Each of them has its advantages or disadvantages for each kind of problem, but to discuss them is far beyond the scope of this course. We will just try them with their basic configuration and keep the one that best fits our case.

#### Linear regression
This is the most basic regression model. The algorithm will just try to minimize the error in a function like $Y=aX+b$, where $Y$ is the target and $X$ the vector of features.

In [ ]:
from sklearn.linear_model import LinearRegression
x, y = datasets.make_regression(n_samples=30, n_features=1, noise=10)

#y=y**2
model = LinearRegression().fit(x,y)
fig,ax = plt.subplots(figsize=(9, 8))
ax.scatter(x, y)
ax.plot(x,model.predict(x), "r")
plt.show()

#### Decision Tree
This model is based on a tree structure in which each node represents a decision and each leaf an output. https://en.wikipedia.org/wiki/Decision_tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
x_test= np.linspace(x.min(),x.max(), 100)[:, np.newaxis]

model = DecisionTreeRegressor().fit(x,y)
fig,ax = plt.subplots(figsize=(9, 8))
ax.scatter(x, y)
ax.plot(x_test ,model.predict(x_test),"r")

plt.show()

#### Random Forest
This model belongs to a category called ensemble models, because it is made of the aggregation of several models. In the case of random forest, we will have a set of decision trees which will calculate an output, then the output of the random forest will be the average of those outputs.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100).fit(x,y)
plt.figure(figsize=(9, 8))
plt.scatter(x, y)
plt.plot(x_test ,model.predict(x_test), "r")
plt.show()

#### K Nearest Neighbors
Given an input, the output of this algorithm will be the average of the $k$ nearest points in the training set.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor(n_neighbors=5).fit(x,y)
plt.figure(figsize=(9, 8))
plt.scatter(x, y)
plt.plot(x_test ,model.predict(x_test), "r")
plt.show()

#### Perceptron
This is the most basic Artificial Neural Network https://en.wikipedia.org/wiki/Artificial_neural_network. Opposite  to the other models shown, this one is a "black box" and we cannot see how the algorithm calculates the output.

Don't worry if you get a warning when running this model.

In [ ]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(max_iter=50000).fit(x,y)
plt.figure(figsize=(9, 8))
plt.scatter(x, y)
plt.plot(x_test ,model.predict(x_test), "r")
plt.show()

### How to evaluate a regression model
We have trained 5 models, and we can guess that some are better than others, but we need a formal way to evaluate the models. There are several ways to evaluate a regression model, all of them based on the __error between the output of the model and the real answer__ when evaluating the test data, so the lower the error, the better the model. Let see some of them:

1. Max error: $max(| y_i - \hat{y}_i |)$

2. Mean absolute error or MAE: $\frac{1}{n_{\text{samples}}} \sum_{i=0}^{n_{\text{samples}}-1} \left| y_i - \hat{y}_i \right|$

3. Mean squared error or MSE: $\frac{1}{n_\text{samples}} \sum_{i=0}^{n_\text{samples} - 1} (y_i - \hat{y}_i)^2$

Don't worry about the math, scikit learn will do everything for us!

Remember about K-Folds Cross Validation when evaluating the models.

In [ ]:
from sklearn.metrics import  mean_absolute_error, mean_squared_error, max_error
models = {"Linear Regression":LinearRegression(), \
          "Decision Tree": DecisionTreeRegressor(), \
          "Random Forest": RandomForestRegressor(n_estimators=100), \
          "Nearest Neighbors": KNeighborsRegressor(n_neighbors=5), \
          "Perceptron": MLPRegressor(max_iter=2000)}

x, y = datasets.make_regression(n_samples=30, n_features=1, noise=10)
k = 3
kf = KFold(n_splits=k,shuffle=True)
for name in models:
    print("Start training models of {}".format(name))
    i = 0
    me_global = []
    mae_global = []
    mse_global = []
    for train, test in kf.split(x):
        i+=1
        x_train, x_test, y_train, y_test = x[train], x[test], y[train], y[test]
        model = models[name].fit(x_train,y_train)
        y_out = model.predict(x_test)
        # We have to provide the real value of the target and the model's output
        me = max_error(y_test, y_out)
        mae = mean_absolute_error(y_test, y_out)
        mse = mean_squared_error(y_test, y_out)
        me_global.append(me)
        mae_global.append(mae)
        mse_global.append(mse)
        print("Iteration {}: me={:.4f} mae={:.4f} mse={:.4f}".format(i,me, mae, mse))
    print("Global: me={:.4f} mae={:.4f} mse={:.4f}\n".format(np.average(me_global), np.average(mae_global), np.average(mse_global)))

### Try it
Now is your turn to create a prediction model. Just after reading the datasets, we have transformed and prepared them in a format suitable for the algorithms we have just seen.

For this case, lets supose that we know the train that will be used in a trip and how many packages will be transported, and we want to predict the duration of the trip. That means that our features are the train_id and the cargo column, and the target is the duration column.

The task is to train some regression models using different algorithms and keep the best one. Then answer the following questions:

- What is the best algorithm?
- What measure have you used to select the best model?
- Do you think that the best model is a "good" regression model? Why?
- Can you think of a variable that we can add to our features to improve the result?

In [ ]:
# Code here!

x = trip_summary[["train_id_encoded", "cargo"]].values
y = trip_summary.trip_duration.values


In [ ]:
# To evaluate this task, you will be asked to run your model in a given situation. 
# For example, the train JITAR has a scheduled trip on 2021-01-23 at 10:00:00, its cargo will be 50 packages. How long will it take to complete the trip?

# Suppose that your model is called "model" and your input is a vector of [train_id, cargo]. Then, you will have to run the following code:
# model.predict([train_id_encoder.transform(["JITAR"])[0],50])

## Classification
Another common task in ML is classification; in this case the target will be a class or a categorical variable, like in the iris dataset.

When talking about classifications task there are two possibilities: the target can only be positive or negative (binary classification) or the target can be any class of a set of categories (multi class).

In [ ]:
x, y = datasets.make_classification(n_samples=300, n_features=2,n_classes=4,n_redundant=0,n_clusters_per_class=1)
plt.figure(figsize=(9, 8))
plt.scatter(x[:,0], x[:,1],c=y)
plt.show()

### Some popular models
Like in the case of regression there are many algorithms for classification. Many of them have implementations for both regression and classification. 

In the following plots, the decision regions will be plotted. Those are the regions that the model will use to perform the classification, an input will be classified depending on which region it appears.

#### Logistic regression
Don't be misleading with the regression word, this model can only be used for classification.

In [ ]:
from sklearn.linear_model import LogisticRegression

x, y = datasets.make_classification(n_samples=300, n_features=2,n_classes=3,n_redundant=0,n_clusters_per_class=1)

# For decision regions
x_min, x_max = x[:, 0].min() - 1, x[:, 0].max() + 1
y_min, y_max = x[:, 1].min() - 1, x[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))

model = LogisticRegression(multi_class="auto", solver="lbfgs").fit(x,y)
plt.figure(figsize=(9, 8))
plt.scatter(x[:,0], x[:,1],c=y)

Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.4)
plt.show()

#### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier().fit(x,y)
plt.figure(figsize=(9, 8))
plt.scatter(x[:,0], x[:,1],c=y)

Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.4)
plt.show()

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100).fit(x,y)
plt.figure(figsize=(9, 8))
plt.scatter(x[:,0], x[:,1],c=y)

Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.4)
plt.show()

#### K Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5).fit(x,y)
plt.figure(figsize=(9, 8))
plt.scatter(x[:,0], x[:,1],c=y)

Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.4)
plt.show()

#### Perceptron

In [ ]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(max_iter=2000).fit(x,y)
plt.figure(figsize=(9, 8))
plt.scatter(x[:,0], x[:,1],c=y)

Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.4)
plt.show()

### How to evaluate a classification model
In this case the output is a class or a label, so we can't calculate the error, instead we will calculate a ratio between the number of times that the output is correct and another count of the outputs, the total number or the number of wrong outputs for example. For example, the most intuitive way to evaluate a classification model is the accuracy : How many times my model guess the correct answer over all the anwsers?. But, even if this can be useful in some cases, in practice, it is not an informative measure.

### Confusion matrix
This is the main tool for evaluating a classification model https://en.wikipedia.org/wiki/Confusion_matrix. It is defined as:

"By definition a confusion matrix $C$ is such that $C_{i,j}$ is equal to the number of observations known to be in group $i$ but predicted to be in group $j$."

In other words, it is a matrix where columns represent the true class and rows represent the output class. With it we can know how many samples have been correctly classified and which classes can be confused.


In the case of binary classification, the matrix has only 2 columns and 2 rows, and each of the cells has a proper name.

There are some metrics obtained from these cells:
1. Precision: the ratio between positive predicted and the total of positive samples $\frac{TP}{TP+FP}$
2. Accuracy: the ratio of correctly predicted over the total of samples $\frac{TP+TN}{TP+FP+TN+FN}$
3. Recall: the ratio of correctly predicted over all predicted as positive $\frac{TP}{TP+FN}$
4. F1 score: harmonic mean of precision and recall $\frac{Precision*Recall}{Precision+Recall}=\frac{2*TP}{2*TP+FP+FN}$

Usually more complex metrics are used, but for this course the F1 score is enough. And again, don't worry, scikit learn includes all these calculations.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score

models = {"Logistic Regression":LogisticRegression(multi_class="auto", solver="lbfgs", max_iter=2000), \
          "Decision Tree": DecisionTreeClassifier(), \
          "Random Forest": RandomForestClassifier(n_estimators=100), \
          "Nearest Neighbors": KNeighborsClassifier(n_neighbors=5), \
          "Perceptron": MLPClassifier(max_iter=2000)}

x, y = datasets.make_classification(n_samples=300)

for name in models:
    print("Start training models of {}".format(name))
    i = 0
    f1_global = []
    for train, test in kf.split(x):
        i+=1
        x_train, x_test, y_train, y_test = x[train], x[test], y[train], y[test]
        model = models[name].fit(x_train,y_train)
        y_out = model.predict(x_test)
        # We have to provide the real value of the target and the model's output
        f1_global.append(f1_score(y_test, y_out, average="weighted", zero_division=0))
        print(confusion_matrix(y_test, y_out))
        print(classification_report(y_test, y_out))
    print("Global: f1={:.4f}\n".format(np.average(f1_global)))

### Try it
In this case, lets suppose that the sensor which reads the id of a train when it arrives to the last station is malfunctioning. However, we are still able to know how many packages were transported and the duration of the trip. We want to create a model that, given the number of packages and the duration of the trip, can predict the train id.

- What is the best algorithm?
- What is the average F1 score for that algorithm?

Some algorithms provides a way to estimate the importance of each feature in the classification. This is useful to know which features are more important for the model and which are not. Tree based models (Decision Tree and Random Forest) have this feature, we can access it with the attribute feature_importances_. The higher the value, the more important the feature is. In this case, which is the most important feature?

In [ ]:
# Code here!

In [ ]:
# To evaluate this task, you will be asked to run your model in a given situation.
# For example, a train has arrive to the last station. Its cargo was 30 packages. 
# It started its trip on 2021-01-23 at 10:00:00 and arived to the last station on 2021-01-23 at 18:00:00
# What is the train's id?